# This is a TF Estimator end-to-end baseline solution

For local run

Tested with

~~~~
numpy==1.13.3
scipy==0.19.1
tensorflow-gpu==1.4.0
tqdm
~~~~

I want to show usage of Estimators with custom python datagenerators.

Detailed documentation you can find at https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator

I also recommend to read source code https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/estimator/estimator.py

Suppose we have following project structure:


```
.
├── data
│   ├── test            # extracted
│   │   └── audio          # all test
│   ├── test.7z         # downloaded
│   ├── train           # extracted
│   │   ├── audio          # folder with all train command/file.wav
│   │   ├── LICENSE
│   │   ├── README.md
│   │   ├── testing_list.txt
│   │   └── validation_list.txt
│   └── train.7z         # downloaded
├── kernel.ipynb      # this ipynb  
└── model-k           # folder for model, checkpoints, logs and submission.csv
```

In [1]:
DATADIR = '../data' # unzipped train and test data
OUTDIR = './model-k' # just a random name
# Data Loading
import os
import re
from glob import glob


POSSIBLE_LABELS = 'yes no up down left right on off stop go silence unknown'.split()
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}


def load_data(data_dir):
    """ Return 2 lists of tuples:
    [(class_id, user_id, path), ...] for train
    [(class_id, user_id, path), ...] for validation
    """
    # Just a simple regexp for paths with three groups:
    # prefix, label, user_id
    pattern = re.compile("(.+\/)?(\w+)\/([^_]+)_.+wav")
    all_files = glob(os.path.join(data_dir, 'train/audio/*/*wav'))

    with open(os.path.join(data_dir, 'train/validation_list.txt'), 'r') as fin:
        validation_files = fin.readlines()
    valset = set()
    for entry in validation_files:
        r = re.match(pattern, entry)
        if r:
            valset.add(r.group(3))

    possible = set(POSSIBLE_LABELS)
    train, val = [], []
    for entry in all_files:
        r = re.match(pattern, entry)
        if r:
            label, uid = r.group(2), r.group(3)
            if label == '_background_noise_':
                label = 'silence'
            if label not in possible:
                label = 'unknown'

            label_id = name2id[label]

            sample = (label_id, uid, entry)
            if uid in valset:
                val.append(sample)
            else:
                train.append(sample)

    print('There are {} train and {} val samples'.format(len(train), len(val)))
    return train, val

trainset, valset = load_data(DATADIR)

There are 57929 train and 6798 val samples


Let me introduce pythonic datagenerator. It is just a python/numpy/... function **without tf** that yields dicts such that

```
{
  'x': np.array(...),
  'str_key': np.string_(...),
  'label': np.int32(...),
}
```

Be sure, every value in this dict has .dtype method.


In [3]:
import numpy as np
from scipy.io import wavfile

def data_generator(data, params, mode='train'):
    def generator():
        if mode == 'train':
            np.random.shuffle(data)
        # Feel free to add any augmentation
        for (label_id, uid, fname) in data:
            try:
                _, wav = wavfile.read(fname)
                wav = wav.astype(np.float32) / np.iinfo(np.int16).max

                L = 16000  # be aware, some files are shorter than 1 sec!
                if len(wav) < L:
                    continue
                # let's generate more silence!
                samples_per_file = 1 if label_id != name2id['silence'] else 20
                for _ in range(samples_per_file):
                    if len(wav) > L:
                        beg = np.random.randint(0, len(wav) - L)
                    else:
                        beg = 0
                    yield dict(
                        target=np.int32(label_id),
                        wav=wav[beg: beg + L],
                    )
            except Exception as err:
                print(err, label_id, uid, fname)

    return generator

Suppose, we have spectrograms and want to write feature extractor that produces logits.

Let's write some simple net, treat sound as a picture.

**Spectrograms** (input x) have shape ```(batch_size, time_frames, freq_bins, 2)```.

**Logits** is a tensor with shape ```(batch_size, num_classes)```.

In [5]:
import tensorflow as tf
from tensorflow.contrib import layers

def baseline(x, params, is_training):
    x = layers.batch_norm(x, is_training=is_training)
    for i in range(4):
        x = layers.conv2d(
            x, 16 * (2 ** i), 3, 1,
            activation_fn=tf.nn.elu,
            normalizer_fn=layers.batch_norm if params.use_batch_norm else None,
            normalizer_params={'is_training': is_training}
        )
        x = layers.max_pool2d(x, 2, 2)

    # just take two kind of pooling and then mix them, why not :)
    mpool = tf.reduce_max(x, axis=[1, 2], keep_dims=True)
    apool = tf.reduce_mean(x, axis=[1, 2], keep_dims=True)

    x = 0.5 * (mpool + apool)
    # we can use conv2d 1x1 instead of dense
    x = layers.conv2d(x, 128, 1, 1, activation_fn=tf.nn.elu)
    x = tf.nn.dropout(x, keep_prob=params.keep_prob if is_training else 1.0)
    
    # again conv2d 1x1 instead of dense layer
    logits = layers.conv2d(x, params.num_classes, 1, 1, activation_fn=None)
    return tf.squeeze(logits, [1, 2])

We need to write a model handler for three regimes:

* train
* eval
* predict

Loss function, train_op, additional metrics and summaries should be defined.

Also, we need to convert sound waveform into spectrograms (we could do it with numpy/scipy/librosa in data generator, but TF has new signal processing API)

In [6]:
from tensorflow.contrib import signal

# features is a dict with keys: tensors from our datagenerator
# labels also were in features, but excluded in generator_input_fn by target_key

def model_handler(features, labels, mode, params, config):
    # Im really like to use make_template instead of variable_scopes and re-usage
    extractor = tf.make_template(
        'extractor', baseline,
        create_scope_now_=True,
    )
    # wav is a waveform signal with shape (16000, )
    wav = features['wav']
    # we want to compute spectograms by means of short time fourier transform:
    specgram = signal.stft(
        wav,
        400,  # 16000 [samples per second] * 0.025 [s] -- default stft window frame
        160,  # 16000 * 0.010 -- default stride
    )
    # specgram is a complex tensor, so split it into abs and phase parts:
    phase = tf.angle(specgram) / np.pi
    # log(1 + abs) is a default transformation for energy units
    amp = tf.log1p(tf.abs(specgram))
    
    x = tf.stack([amp, phase], axis=3) # shape is [bs, time, freq_bins, 2]
    x = tf.to_float(x)  # we want to have float32, not float64

    logits = extractor(x, params, mode == tf.estimator.ModeKeys.TRAIN)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        # some lr tuner, you could use move interesting functions
        def learning_rate_decay_fn(learning_rate, global_step):
            return tf.train.exponential_decay(
                learning_rate, global_step, decay_steps=10000, decay_rate=0.99)

        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=params.learning_rate,
            optimizer=lambda lr: tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True),
            learning_rate_decay_fn=learning_rate_decay_fn,
            clip_gradients=params.clip_gradients,
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

        specs = dict(
            mode=mode,
            loss=loss,
            train_op=train_op,
        )

    if mode == tf.estimator.ModeKeys.EVAL:
        prediction = tf.argmax(logits, axis=-1)
        acc, acc_op = tf.metrics.mean_per_class_accuracy(
            labels, prediction, params.num_classes)
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        specs = dict(
            mode=mode,
            loss=loss,
            eval_metric_ops=dict(
                acc=(acc, acc_op),
            )
        )

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'label': tf.argmax(logits, axis=-1),  # for probability just take tf.nn.softmax()
            'sample': features['sample'], # it's a hack for simplicity
        }
        specs = dict(
            mode=mode,
            predictions=predictions,
        )
    return tf.estimator.EstimatorSpec(**specs)


def create_model(config=None, hparams=None):
    return tf.estimator.Estimator(
        model_fn=model_handler,
        config=config,
        params=hparams,
    )

Define some params. Move model hyperparams (optimizer, extractor, num of layers, activation fn, ...) here

In [7]:
params=dict(
    seed=2018,
    batch_size=64,
    keep_prob=0.5,
    learning_rate=1e-3,
    clip_gradients=15.0,
    use_batch_norm=True,
    num_classes=len(POSSIBLE_LABELS),
)

hparams = tf.contrib.training.HParams(**params)
os.makedirs(os.path.join(OUTDIR, 'eval'), exist_ok=True)
model_dir = OUTDIR

run_config = tf.contrib.learn.RunConfig(model_dir=model_dir)

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.


**Let's run training!**

In [8]:
# it's a magic function :)
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn
            
train_input_fn = generator_input_fn(
    x=data_generator(trainset, hparams, 'train'),
    target_key='target',  # you could leave target_key in features, so labels in model_handler will be empty
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)

val_input_fn = generator_input_fn(
    x=data_generator(valset, hparams, 'val'),
    target_key='target',
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)
            

def _create_my_experiment(run_config, hparams):
    exp = tf.contrib.learn.Experiment(
        estimator=create_model(config=run_config, hparams=hparams),
        train_input_fn=train_input_fn,
        eval_input_fn=val_input_fn,
        train_steps=10000, # just randomly selected params
        eval_steps=200,  # read source code for steps-epochs ariphmetics
        train_steps_per_iteration=1000,
    )
    return exp

tf.contrib.learn.learn_runner.run(
    experiment_fn=_create_my_experiment,
    run_config=run_config,
    schedule="continuous_train_and_eval",
    hparams=hparams)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Use tf.estimator.train_and_evaluate.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5aa91974a8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './model-k'}
Instructions for updating:
Please switch to tf.estimator.train_

/home/cs/anaconda3/lib/python3.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


INFO:tensorflow:global_step/sec: 24.2846
INFO:tensorflow:loss = 1.3337922, step = 300 (4.118 sec)
INFO:tensorflow:global_step/sec: 24.1337
INFO:tensorflow:loss = 1.0936182, step = 400 (4.143 sec)
INFO:tensorflow:global_step/sec: 24.0979
INFO:tensorflow:loss = 1.1033119, step = 500 (4.150 sec)
INFO:tensorflow:global_step/sec: 23.6984
INFO:tensorflow:loss = 1.1183573, step = 600 (4.220 sec)
INFO:tensorflow:global_step/sec: 23.6454
INFO:tensorflow:loss = 0.98763216, step = 700 (4.229 sec)
INFO:tensorflow:global_step/sec: 23.4584
INFO:tensorflow:loss = 0.756913, step = 800 (4.262 sec)
INFO:tensorflow:global_step/sec: 23.5045
INFO:tensorflow:loss = 0.72922885, step = 900 (4.255 sec)
INFO:tensorflow:Saving checkpoints for 1000 into ./model-k/model.ckpt.
INFO:tensorflow:Loss for final step: 0.5589514.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation

INFO:tensorflow:global_step/sec: 24.4698
INFO:tensorflow:loss = 0.42106393, step = 3200 (4.087 sec)
INFO:tensorflow:global_step/sec: 24.3694
INFO:tensorflow:loss = 0.43505046, step = 3300 (4.104 sec)
INFO:tensorflow:global_step/sec: 24.5009
INFO:tensorflow:loss = 0.49810016, step = 3400 (4.082 sec)
INFO:tensorflow:global_step/sec: 24.3524
INFO:tensorflow:loss = 0.3980983, step = 3500 (4.107 sec)
INFO:tensorflow:global_step/sec: 24.4184
INFO:tensorflow:loss = 0.3465563, step = 3600 (4.095 sec)
INFO:tensorflow:global_step/sec: 24.4238
INFO:tensorflow:loss = 0.34246966, step = 3700 (4.095 sec)
INFO:tensorflow:global_step/sec: 24.2272
INFO:tensorflow:loss = 0.5390545, step = 3800 (4.127 sec)
INFO:tensorflow:global_step/sec: 24.2886
INFO:tensorflow:loss = 0.35721043, step = 3900 (4.117 sec)
INFO:tensorflow:Saving checkpoints for 4000 into ./model-k/model.ckpt.
INFO:tensorflow:Loss for final step: 0.3501766.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Calling model_fn.
INFO:tensorf

INFO:tensorflow:global_step/sec: 24.3472
INFO:tensorflow:loss = 0.29394424, step = 6700 (4.107 sec)
INFO:tensorflow:global_step/sec: 24.3959
INFO:tensorflow:loss = 0.13413703, step = 6800 (4.099 sec)
INFO:tensorflow:global_step/sec: 24.2814
INFO:tensorflow:loss = 0.14225996, step = 6900 (4.118 sec)
INFO:tensorflow:Saving checkpoints for 7000 into ./model-k/model.ckpt.
INFO:tensorflow:Loss for final step: 0.14366898.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-12T20:55:23Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model-k/model.ckpt-7000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [20/200]
INFO:tensorflow:Evaluation [40/200]
INFO:tensorflow:Evaluation [60/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [100/200]
INFO:tensorflow:Evaluation [120/200]
INF

INFO:tensorflow:Evaluation [20/200]
INFO:tensorflow:Evaluation [40/200]
INFO:tensorflow:Evaluation [60/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [100/200]
INFO:tensorflow:Evaluation [120/200]
INFO:tensorflow:Evaluation [140/200]
INFO:tensorflow:Evaluation [160/200]
INFO:tensorflow:Evaluation [180/200]
INFO:tensorflow:Evaluation [200/200]
INFO:tensorflow:Finished evaluation at 2019-06-12-20:57:48
INFO:tensorflow:Saving dict for global step 10000: acc = 0.7688172, global_step = 10000, loss = 0.27326548
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: ./model-k/model.ckpt-10000
INFO:tensorflow:Stop training model as max steps reached


({'acc': 0.7688172, 'loss': 0.27326548, 'global_step': 10000}, [])

Now we want to predict testset and make submission file.

1. Create datagenerator and input_function
2. Load model
3. Iterate over predictions and store results

In [12]:
from tqdm import tqdm
# now we want to predict!
paths = glob(os.path.join(DATADIR, 'test/audio/*wav'))



def test_data_generator(data):
    def generator():
        for path in data:
            _, wav = wavfile.read(path)
            wav = wav.astype(np.float32) / np.iinfo(np.int16).max
            fname = os.path.basename(path)
            yield dict(
                sample=np.string_(fname),
                wav=wav,
            )

    return generator

test_input_fn = generator_input_fn(
    x=test_data_generator(paths),
    batch_size=hparams.batch_size, 
    shuffle=False, 
    num_epochs=1,
    queue_capacity= 10 * hparams.batch_size, 
    num_threads=1,
)

model = create_model(config=run_config, hparams=hparams)
it = model.predict(input_fn=test_input_fn)


# last batch will contain padding, so remove duplicates
submission = dict()
for t in tqdm(it):
    fname, label = t['sample'].decode(), id2name[t['label']]
    submission[fname] = label

with open(os.path.join(model_dir, 'submission.csv'), 'w') as fout:
    fout.write('fname,label\n')
    for fname, label in submission.items():
        fout.write('{},{}\n'.format(fname, label))

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5a0fc9e908>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './model-k'}


0it [00:00, ?it/s]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model-k/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


158560it [01:12, 2187.44it/s]


### About tf.Estimators

#### Pros:

* no need to control Session
* datagenerator feeds model via queues without explicit queue coding :)
* you could naturaly export models into production

#### Cons:

* it's very hard to debug computational graph (use tf.add_check_numerics() and tf.Print in case of problems)
* boilerplate code
* need to read source code for making interesting things

#### Conclusion: Estimator is a nice abstraction with some boilerplate code :)

### About Speech Recognition Challenge:

You could start from this end-to-end ipynb, improving several functions for much better results.

May the gradient flow be with you.